# Table of Contents
* [Build dataset](#Build-dataset)


In [1]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [64]:
import glob

In [63]:
from sun_utils.build_ds import build_dataset

# Build dataset

In [61]:
"""
design draft v3:
{
    image: {
        name
        scene
        layout
        intrinsics
        detector
        filenames: {
            image
            fullres
            depth
        }
        segmentation_arr
        objects : {
            obj_id: {
                label
                globalID
                2D: {
                    polygon: []
                    rectangle: [
                },
                3D: {
                    depth
                },
                phys_props: {
                    
                }
            }
            .
            .
            .
        }
    }
}
"""
None

In [65]:
all_ds_files = glob.glob('./SUNRGBD/**', recursive=True)

In [66]:
sun_ds_scaf = build_dataset(all_ds_files)

100%|██████████| 10331/10331 [00:03<00:00, 3245.91it/s]


In [67]:
single_image_anno = list(sun_ds_scaf.values())[2500]

In [69]:
# %page single_image_anno

# Hide

In [71]:
# fnl = [anno['2D']['fileList'] for anno in sun_ds_scaf.values() if '2D' in anno.keys() and len(anno['2D']['fileList']) == 1]

# pd.Series([len(anno['2D']['fileList']) for anno in sun_ds_scaf.values() if '2D' in anno.keys()]).value_counts()

# cflct = [anno['2D']['conflictList'] for anno in sun_ds_scaf.values() if '2D' in anno.keys() and len(anno['2D']['fileList']) == 1]